In [1]:
import pandas as pd
import torch
from torch_rechub.models.multi_task import single_task
from torch_rechub.trainers import MTLTrainer
from torch_rechub.basic.features import DenseFeature, SparseFeature
from torch_rechub.utils.data import DataGenerator

In [ ]:
model_name = 'single_task'
epoch = 10
learning_rate = 1e-3
batch_size = 32768
weight_decay = 1e-5
save_dir = './save_dir'
seed = 2024
gpus = None
torch.manual_seed(seed)

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [4]:
task_select = 0  #0: cvr, 1:ctr
def get_ali_ccp_data_dict(model_name, task_select = task_select, data_path="data/ali-ccp"):
    df_train = pd.read_csv(data_path + '/ali_ccp_train.csv')
    df_val = pd.read_csv(data_path + '/ali_ccp_val.csv')
    df_test = pd.read_csv(data_path + '/ali_ccp_test.csv')
    print("train : val : test = %d %d %d" % (len(df_train), len(df_val), len(df_test)))
    train_idx, val_idx = df_train.shape[0], df_train.shape[0] + df_val.shape[0]
    data = pd.concat([df_train, df_val, df_test], axis=0)
    #task 1 (as cvr): main task, purchase prediction
    #task 2(as ctr): auxiliary task, click prediction
    data.rename(columns={'purchase': 'cvr_label', 'click': 'ctr_label'}, inplace=True)
    data["ctcvr_label"] = data['cvr_label'] * data['ctr_label']

    col_names = data.columns.values.tolist()
    dense_cols = ['D109_14', 'D110_14', 'D127_14', 'D150_14', 'D508', 'D509', 'D702', 'D853']
    sparse_cols = [col for col in col_names if col not in dense_cols and col not in ['cvr_label', 'ctr_label', 'ctcvr_label']]
    print("sparse cols:%d dense cols:%d" % (len(sparse_cols), len(dense_cols)))
    #define dense and sparse features
    if model_name == "ESMM":
        label_cols = ['cvr_label', 'ctr_label', "ctcvr_label"]  #the order of 3 labels must fixed as this
        #ESMM only for sparse features in origin paper
        item_cols = ['129', '205', '206', '207', '210', '216']  #assumption features split for user and item
        user_cols = [col for col in sparse_cols if col not in item_cols]
        user_features = [SparseFeature(col, data[col].max() + 1, embed_dim=16) for col in user_cols]
        item_features = [SparseFeature(col, data[col].max() + 1, embed_dim=16) for col in item_cols]
        x_train, y_train = {name: data[name].values[:train_idx] for name in sparse_cols}, data[label_cols].values[:train_idx]
        x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in sparse_cols}, data[label_cols].values[train_idx:val_idx]
        x_test, y_test = {name: data[name].values[val_idx:] for name in sparse_cols}, data[label_cols].values[val_idx:]
        return user_features, item_features, x_train, y_train, x_val, y_val, x_test, y_test
    else:
        label_cols = ['cvr_label', 'ctr_label']  #the order of labels can be any
        
        label_col = [label_cols[task_select]]
        
        used_cols = sparse_cols + dense_cols
        features = [SparseFeature(col, data[col].max()+1, embed_dim=4)for col in sparse_cols] \
                   + [DenseFeature(col) for col in dense_cols]
        x_train, y_train = {name: data[name].values[:train_idx] for name in used_cols}, data[label_col].values[:train_idx]
        x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in used_cols}, data[label_col].values[train_idx:val_idx]
        x_test, y_test = {name: data[name].values[val_idx:] for name in used_cols}, data[label_col].values[val_idx:]
        return features, x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
if model_name == "single_task":
    task_type = "classification"
    features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
    # Set model hyperparameters
    model = single_task(features, task_type, hidden_units = {"dims": [256,64]}, tower_params_list = {"dims": [32]})
dg = DataGenerator(x_train, y_train)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)

In [7]:
mtl_trainer = MTLTrainer(model, task_types=[task_type], optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=10, device=device, gpus=gpus, model_path=save_dir)
# started training
file_path = 'path/to/save/output/result_{}_{}.txt'.format(model_name, task_select)
mtl_trainer.fit(train_dataloader, val_dataloader, mode = 'mark1', seed = 'mark2', file_path = file_path)

In [ ]:
auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epo = ['test_result']
my_list = epo + auc
my_list = ', '.join(map(str, my_list))

try:
    with open(file_path, 'a') as file:
        file.write(my_list + '\n')
except Exception as e:  
    print(f"An error occurred while adding to the file: {e}")